# **Hands-On TMS-EEG Preprocessing**

### [Matteo De Matola](https://github.com/matteo-d-m) 

This notebook contains code and explanations for the hands-on TMS-EEG preprocessing activity, [Brain Stimulation & Multimodal Electrophysiological Recording](https://unitn.coursecatalogue.cineca.it/insegnamenti/2025/50512_653501_96292/2011/50513/10168?annoOrdinamento=2011&coorte=2024) course, [Master's degree in Cognitive Science](https://corsi.unitn.it/en/cognitive-science), University of Trento. The aim of this activity is to introduce the main steps of a typical TMS-EEG preprocessing pipeline, explaining their conceptual background alongside their Python implementation. 

The data used throughout the notebook come from one condition and one subject of [Zazio et al. (2021)](https://www.sciencedirect.com/science/article/pii/S1388245721006714). The data are publicly available and freely reusable through [G-Node Gin](https://gin.g-node.org/) at [this](https://gin.g-node.org/AgneseZazio/ZazioMiniussiBortoletto2021) link under a [Creative Commons CC0 1.0 Universal license](https://creativecommons.org/publicdomain/zero/1.0/deed.en). 

The preprocessing pipeline implemented in this notebook is loosely inspired by the one described in Zazio et al. (2021) under section "_2.3.1. TMS-EEG preprocessing_". The original pipeline has been largely preserved for consistency, but also adapted to the constraints of a short hands-on educational activity. The most relevant modification is the omission of the [SOUND](https://www.sciencedirect.com/science/article/abs/pii/S1053811917308480) and [SSP-SIR](https://www.sciencedirect.com/science/article/abs/pii/S1053811916301495) algorithms, which could not be treated with clarity in the time at hand.

This notebook has no pretense of endorsing one particular pipeline as the most effective or most elegant. The aim is 100% educational and the target audience are absolute beginners. Therefore, the preprocessing steps included in the pipeline are merely the most common, which cannot be ignored by an introductory presentation.

# **Introduction**

Given a signal generated by some system, _preprocessing_ is the act of separating the actual signal from the noise.

In the context of TMS-EEG:

- The _signal_ is the fraction of scalp voltage generated by the cerebral cortex in response to TMS pulses
- The _noise_ is the fraction of scalp voltage generated by any non-cerebral source, including (but not limited to) muscles, eyes, and electronic instrumentation 

Preprocessing is usually a complex process that includes multiple sequential transformations of the data. Such sequence of transformations is a _**preprocessing pipeline**_.

There is no standard preprocessing pipeline for TMS-EEG data (nor for EEG data in general). The problem is significant enough to write papers about it &mdash; see [Bertazzoli et al. (2021)](https://www.sciencedirect.com/science/article/pii/S1053811921005486), [Rogasch et al., 2022](https://www.sciencedirect.com/science/article/pii/S0165027022000218?via%3Dihub), [Brancaccio et al., 2024](https://www.sciencedirect.com/science/article/pii/S1053811924003719). 


Broadly speaking, one could classify preprocessing pipelines in two categories: _conservationist approaches_ and _interventionist approaches_.

- _Conservationist approaches_ tend to conserve the signal as it is acquired, keeping transformations to a minimum at risk of not eliminating noise
    - Assumption: any transformation might delete noise, but it alters the signal in potentially undesirable ways
    - See [Delorme (2023)](https://www.nature.com/articles/s41598-023-27528-0)
- _Interventionist approaches_ tend to act heavily on the signal, possibly including complex algorithmic approaches that can be difficult to interpret
    - Assumption: experimental manipulations of the EEG signal will not be visible until all artifacts have been removed

In practice, people tend to design pipelines that suit their experimental manipulations and the characteristics of their data. 

For example, if TMS is delivered on an area where there are large cranial muscles, data will probably be contaminated by large muscle artifacts. Therefore, it will be reasonable to design a pipeline that acts aggressively on muscle artifacts. 

The remander of this document will treat four preprocessing steps that are found across most TMS-EEG pipelines. These are:

1. Basic preprocessing, including data loading, interpolation of the TMS pulse, and high-pass filtering
2. Independent component analysis
3. Manual artifact rejection
4. Assessing and computing a TEP, including preparatory steps like re-rereferencing and baseline correction

# **0. Import Required Libraries**

The code in the following cell imports the Python libraries that are required to run the rest of the notebook. In other words, it grants the notebook access to code contained in a series of files that were previously written by other people and that you have installed before the start of this activity*. For MATLAB users, this is equivalent to adding a toolbox to your MATLAB path.

---

*_If you have not done the installations, you can follow the instructions [here](https://github.com/coneco-lab/brainstim-multimodal/tree/main?tab=readme-ov-file#installation-instructions) before you start_.

In [ ]:
from pathlib import Path                            # to build a bridge between Python and the filesystem 

import numpy as np                                  # to perform array operations
import matplotlib                                   # to draw plots
matplotlib.use('Qt5Agg')                            # to make the plots interactive 

import mne                                          # to read and manipulate EEG data

from scripts import utils                           # custom functions to shorten the code in this notebook

# **1. Basic Preprocessing**

Basic preprocessing is the process of loading data, inspecting them to understand their shape and content, and applying some basic transformations that set the stage for subsequent, more complex operations.

In a typical TMS-EEG preprocessing pipeline, this would include the following steps:

1. Loading the data

2. Inspecting the data, looking at quantities like:
    - Their shape &mdash; that is, the number of channels and the timelength
    - The acquisition parameters &mdash; for example, their sampling rate
    - Their appearance: are they visibly dirty?
    
3. Adjusting data 
    - If present, drop non-EEG channels (EMG, EOG, ECG)
    - Set channel locations
4. Interpolating the pulse artifact

5. High-pass filtering

## **1.1 Loading Data**

This step amounts merely to reading the data files. 

EEG data files come in a variety of formats, depending on the recording system and the choices made by researchers. The four most common formats are (in no particular order):

1. BrainVision format: `.eeg` + `.vhdr` + `.vmrk` = one single recording
    - Typical of data acquired with Brain Products hardware, which usually ships with the BrainVision Recorder software

2. EEGLAB format: `.set` + `.fdt` = one single recording 
    - Typical of data that were previously analysed or otherwise treated with EEGLAB, the main Matlab-based EEG toolbox
    - Some recording systems (e.g., Bittium NeurOne) have a built-in option to save data in this format to facilitate EEGLAB users

3. Matlab format: `.mat`
    - Typycal of recording systems that run Matlab-based software (e.g., g.tec systems)

4. European Data Format: `.edf` 

MNE-Python has functions to read data in all the formats above, except `.mat` which is probably the least common. 

In this activity we will use data acquired with Brain Products hardware and stored in BrainVision format.

As mentioned above, the BrainVision format distributes information across three different files: 

1. The `.eeg` file, which contains actual EEG time series 
2. The `.vhdr` file (also known as _header file_), which contains important metadata about _how_ the data were acquired &mdash; chiefly, the map between channel labels (e.g., `Fp1`) and their place in the recording hardware (e.g., _pin number one_)
3. The `.vmrk` file (also known as _marker file_), which contains important information about event markers &mdash; that is, _when_ an event like a TMS pulse happened and _how long_ did it last

To read data in BrainVision format, the go-to function is `read_raw_brainvision()`, contained in module `io` (that is, input/output) of the `mne` package.

The code below performes the following operations:

- Define a `Path` object (basically, make the data directory accessible to Python)
- Identify the file of interest
- Read it using `read_raw_brainvision()`
- Assign the result to a Python variable, where EEG data will be represented as a `channels x time` matrix with accompanying metadata

In [ ]:
data_dir = Path("data")
subject_and_condition = "S02C1_M1"

file_of_interest = list(data_dir.glob(pattern=f"{subject_and_condition}.vhdr"))
eeg_data = mne.io.read_raw_brainvision(vhdr_fname=file_of_interest[0])

Note that the file to read is not the  EEG file itself (`.eeg`), but the header file (`.vhdr`). This is because the header contains metadata that are needed to correctly interpret the information from the EEG file, which would otherwise be a meaningless array of numbers. The same goes for the EEGLAB format, where the file to read is the `.fdt` that contains metadata 

After reading the header, `read_raw_brainvision()` proceeds to read the actual data from the corresponding EEG file. This implies that the two filenames **must** be identical (except for the extension), otherwise Python will throw an error.

After reading the data, it is possible to start inspecting them by printing the name of the corresponding Python variable. 

This gives access to a first set of useful information concerning the acquisition set-up, the channels and the filters that were applied to the signal during the recording. 

In [ ]:
eeg_data

As you can see, information is classified as `General`, `Acquisition`, `Channels`, and `Filters`. 

Metadata under the `General` section yield no particular insights. 

Much more interesting is the `Acquisition` section, which describes the temporal characteristics of the data: the duration of the recording, the sampling rate, and the number of timepoints. 

🤔 We shall now pause for a minute and ponder the relationship between these three quantities. We can see that:

- The recording length is expressed in minutes (which are a multiple of seconds)
- The sampling rate is expressed in Hz (that is, samples per second)
- The number of timepoints is expressed in array units &mdash; in other words, it is: 
    - The number of points on the horizontal axis of the array that stores the EEG data
    - The number of columns in the EEG data matrix

<p align="center">
<img src="./files/eeg.png" width="500"/>
</p>


The three quantities (minutes, Hz, array units) have _time_ in common &mdash; therefore, it is possible to convert from one to another with simple operations. This is useful to acquire a deeper undestanding of things and to recover one quantity from the other two (should that become necessary).

The following relationships are particularly useful when you have to [wrangle](https://en.wikipedia.org/wiki/Data_wrangling) with ill-defined data:

$$ \text{Number of Timepoints} = \text{Recording Length in Minutes} \cdot \text{Sampling Rate} $$

$$ \text{Recording Length in Minutes} = \Bigg(\dfrac{\text{Number of Timepoints}}{\text{Sampling Rate}}\Bigg) \cdot \dfrac{1}{60}$$

where $60$ is the number of seconds in a minute. You can check for yourself that the following results are coherent with what you find in the dataset's info:

In [ ]:
NUMBER_OF_TIMEPOINTS = len(eeg_data.times)
SAMPLING_RATE = eeg_data.info["sfreq"]
SECONDS_IN_A_MINUTE = 60

RECORDING_LENGTH_IN_MINUTES = NUMBER_OF_TIMEPOINTS / SAMPLING_RATE / SECONDS_IN_A_MINUTE
print(f"The recording length is: {RECORDING_LENGTH_IN_MINUTES}")

NUMBER_OF_TIMEPOINTS = RECORDING_LENGTH_IN_MINUTES*SAMPLING_RATE*SECONDS_IN_A_MINUTE
print(f"The number of timepoints is: {NUMBER_OF_TIMEPOINTS}")

Metadata under the `Channels` section are also interesting, as they give us information about the number of channels and whether their position was [digitised](https://www.nature.com/articles/s41598-023-30223-9) during the recording. As you can see from the table above, no digitisation was performed. However, the list of channels is available and can be accessed as:

In [ ]:
eeg_data.ch_names

## **1.2 Drop Unwanted Channels**

As you could see from the list above, the dataset contains EEG channels (`Fp1`,`Fpz`,`Fp2`... etc.) as well as non-EEG channels (`EOG` and `FDI`).

Assuming that we are not interested in analysing non-EEG channels, we can drop them. To this end, we need to identify them. This can either be done manually (which would be boring and time-consuming) or automatically. In the latter case, one can exploit the fact that the last character in EEG channel names tends to be an integer number (`Fp1`, `Fp2`, `AF7`...) or a lowercase _z_ (`Fpz`, `Cz`, `Oz`...). The following code does just that: for each channel name, it checks if its last character is an integer. If it is not (that is, if Python throws a `ValueError`), it checks if such character is a _z_. If it is not, the channel must be non-EEG and is thus appended to a list of channels that need be dropped. 

You can check for yourself that the resulting list contains only channels with a non-EEG name.

In [ ]:
channels_to_drop = []

for channel_name in eeg_data.ch_names:
    try:
        int(channel_name[-1])
    except ValueError:
        if channel_name[-1] != "z": 
            channels_to_drop.append(channel_name)
print(f"Channels to drop: {channels_to_drop}")

After identifying the channels to drop, one can actually drop them as follows:

In [ ]:
eeg_data = eeg_data.drop_channels(ch_names=channels_to_drop)

Re-printing dataset info shows that the data now have 70 channels, which is two less than before (as expected):

In [ ]:
eeg_data

## **1.3 Set Channel Locations**

As revealed by its info, the dataset contains no _Head & sensor digitization_ &mdash; in other words, we have no way of knowing the shape of the head and the position of channels on it. 

However, this is fundamental information for some analyses (_source reconstruction_) and visualizations (_topoplots_).

While we will not perform source reconstruction, we will visualize topoplots &mdash; that is, colour-coded scalp voltage maps. 

Therefore, we need to associate each channel to a set of coordinates (called _montage_) that locate them on the head. To this end, MNE has a list of built-in coordinate sets that can be printed as follows:

In [ ]:
mne.channels.get_builtin_montages()

We know from the publication that the authors used a 74-channels Brain Products cap, which should correspond to MNE's `easycap-M1` montage. 

Therefore, we can create a corresponding _montage_ object and print the resulting $x,y,z$ coordinates for each channel and the three fiducial points: nasion, left pre-auricular (LPA) and right pre-auricular (RPA): 

In [ ]:
easycap_m1_montage = mne.channels.make_standard_montage(kind="easycap-M1")
easycap_m1_montage.get_positions()

Having 3D coordinates, we can project them onto a plane and build the following plot:

In [ ]:
easycap_m1_montage.plot()

After having visualized the montage and ascertained that it makes sense, we can apply it to our data as follows: 

In [ ]:
eeg_data.set_montage(montage=easycap_m1_montage,
                     on_missing="ignore")

We are finally ready to visualize the raw data:

In [ ]:
eeg_data.plot()

The next step would be to interpolate the TMS pulse artefact. To this end, we need to locate TMS pulses on the EEG trace. Therefore, we need information about _when_ TMS pulses occurred during the experiment.

This information is given by **event markers**, which are timestamps that locate events of experimental interest on the EEG trace. These timestamps have three defining characteristics:

1. A name: for example, a numeric code like `54` or a short word like `tms`
2. A duration 
3. A time of occurrence 

In the BrainVision data format, marker information are stored in the `.vmrk` file and are enriched by some supplementary information. In fact, they are defined by:

1. An ordinal number
2. A type
3. A name
4. A time of occurrence (in units of data points)
5. A duration (in units of data points)
6. Information about which channels are involved

For example, the following row of the marker file tells us that marker number 200 (`Mk200`) is of type `Stimulus`, its name is `S 54`, it occurred at time point `3183619`, it lasted one instant (`1`), and it affected all channels (in the BrainVision convention, `0` means "all channels"):

`Mk200=Stimulus,S 54,3183619,1,0`

MNE has a built-in function to extract marker information from EEG data structures. 

The function is called `events_from_annotations()` and its goal is to translate events data into a Python-friendly format, consisting of:

1. One array with as many rows as there are events (in our case, 200) and three columns: one for the time of occurrence, one for the affected channels, and one for the event's name
2. One dictionary that associates each event name to a number-only format. This one is of secondary importance for the sake of understanding the event, as the name is arbitrary

In [ ]:
events_from_annotations, events_dict = mne.events_from_annotations(raw=eeg_data)
events_from_annotations = events_from_annotations[2:]

## **1.4. Interpolate the Pulse Artifact: Rationale & Execution**

The TMS pulse leaves a large artifact on EEG traces ([Veniero et al., 2009](https://www.sciencedirect.com/science/article/pii/S1388245709003629), [Freche et al., 2018](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1006177)). 

This artifact is due to the interaction between the EEG recording system and the electric field induced by the pulse, which is orders of magnitude larger than physiological fields. 

The following code creates epochs around TMS pulses, averages them, and plots the resulting TEP for the sake of visualizing the pulse artifact. 

In [ ]:
pre_interpolation_epochs = mne.Epochs(raw=eeg_data,
                                      events=events_from_annotations,
                                      tmin=-1.1,
                                      tmax=0.5,
                                      baseline=None)
pre_interpolation_epochs_tep = pre_interpolation_epochs.average()
pre_interpolation_epochs_tep.plot();
del pre_interpolation_epochs

As you can see, the EEG traces are dominated by a large spike at the time of the TMS pulse &mdash; so large that everything else appears flat. That is the pulse artifact. 

There is currently no way to recover physiological signals from a pulse artifact. One can only delete the artifact and interpolate the empty window underneath. 

Interpolation is the act of estimating a function in some interval, given values from a preceding and a following interval. 

In other words, interpolation answers the question: "**Given the pre- and post-pulse EEG, what should the EEG around the pulse look like?**"

<p align="center">
<img src="./files/interpolation.png" width="1000"/>
</p>

There exist a wealth of interpolation methods. Historically, the TMS-EEG community has adopted the following: 

- Linear interpolation 
    - Fits a linear function in the artifact window
    - Implausible: the EEG is seldom linear
- Moving average interpolation 
    - Replaces the artifact with a moving average that starts before the pulse
    - Reasonable approach but suboptimal results
- Cubic spline interpolation 
    - Fits a cubic function between each pair of contiguous `(time, voltage)` points in the interval of interest
    - Reasonable approach and satisfying results &mdash; current state of the art  

The following code loads data that were were previously interpolated with a cubic spline between 2 ms pre-pulse and 5 ms post-pulse, then segments the signal into epochs and computes the corresponding TEP for visualization.

In [ ]:
post_interpolation_eeg = mne.io.read_raw(fname="data/post_2_5_interpolation_eeg.fif",
                                         preload=True)
post_interpolation_epochs = mne.Epochs(raw=post_interpolation_eeg,
                                       events=events_from_annotations,
                                       tmin=-1.1,
                                       tmax=0.5,
                                       baseline=None)
post_interpolation_tep = post_interpolation_epochs.average()
post_interpolation_tep.plot();

## **1.5. High-Pass Filtering: Rationale & Execution**


Filtering is the convolution* of a signal with a kernel whose characteristics can highlight a pattern of interest while suppressing other patterns.

For example, high-pass filtering with a cut frequency (or _threshold_) $\tau$ is the convolution of the signal with a kernel that highlights  high-frequency components, where "high" means "larger than $\tau$". 

We are interested in high-pass filtering the data to discard all frequencies below 0.1 Hz, preserving others. Oscillations at very low frequencies (so-called _slow drifts_) are usually due to slow-cycling local currents that are generated at the scalp by processes like sweating or the exchange of ions between the electrode and the electrode gel, so they are not interesting for scientific purposes and we can filter them out.

---

*_Convolution is a mathematical operation that cannot be fully understood without some basics of linear algebra and calculus. 
However, people without a quantitative background can get a working understanding of convolution from resources like [this article](https://betterexplained.com/articles/intuitive-convolution/) at BetterExplained or the appropriate chapters from the book "Analyzing Neural Time Series Data", by Mike X Cohen (MIT Press) (available from [BUR &mdash; Rovereto University Library](https://www.biblioteca.unitn.it/en/bur-rovereto-university-library)). 
Finally, [de Cheveigné & Nelken (2019)](https://www.sciencedirect.com/science/article/pii/S0896627319301746) provide a good introduction to filters and their use in EEG._

The code below applies a high-pass filter to the data with a cut frequency of 0.1 Hz. This returns a NumPy array, which is a simple container of numbers without the attributes of an MNE-Python object. Therefore, the code proceeds to create a Raw object using `mne.io.RawArray`. Finally, it cuts the data into epochs and calculates the TEP for visualization.

In [ ]:
post_filtering_eeg = mne.filter.filter_data(data=post_interpolation_eeg.get_data(),
                                            sfreq=post_interpolation_eeg.info["sfreq"],
                                            l_freq=0.1,
                                            h_freq=None,
                                            method="iir",
                                            iir_params=None,
                                            copy=True,
                                            phase="zero")
post_filtering_eeg = mne.io.RawArray(data=post_filtering_eeg,
                                     info=post_interpolation_eeg.info)
post_filtering_epochs = mne.Epochs(raw=post_filtering_eeg,
                                   events=events_from_annotations,
                                   tmin=-1.1,
                                   tmax=0.5,
                                   baseline=None)
post_filtering_tep = post_filtering_epochs.average()
post_filtering_tep.plot();

# **2. Independent Component Analysis (ICA)**

## **2.1. Rationale**

Independent Component Analysis (ICA) is a mathematical technique to decompose a signal into a set of factors that respect the following constraints:

- They are statistically independent of one another &mdash; that is, observing one does not influence the likelihood of observing the other
- The original signal can be reconstructed as a combination of said factors*

ICA has been used for EEG analyses since the mid 1990s ([Makeig et al., 1996](https://proceedings.neurips.cc/paper_files/paper/1995/file/754dda4b1ba34c6fa89716b85d68532b-Paper.pdf)), with the goal of isolating brain from non-brain signals &mdash; in particular, eye and muscle noise. 

---

*_Another constraint (which must be mentioned for completeness, but is less important for a first grasp of ICA) is that the probability of observing any given value of the factors should follow a non-Gaussian distribution. Interested students can refer to [Makeig et al. (1996)](https://proceedings.neurips.cc/paper_files/paper/1995/file/754dda4b1ba34c6fa89716b85d68532b-Paper.pdf) or [Shlens (2014)](https://arxiv.org/abs/1404.2986) for a deeper treatment of ICA mathematics and related assumptions. Otherwise, the book "Independent Component Analysis: A Tutorial Introduction" by James V. Stone (MIT Press) is available from [BUR &mdash; Rovereto University Library](https://www.biblioteca.unitn.it/en/bur-rovereto-university-library)_

Typically, EEG researchers use ICA in a **five-step** process:

1. Fit an ICA model to the data (that is, run code that factors the signal into independent components)

2. Visualize the resulting components with topoplots and voltage time series
    - Topoplots are colour-coded scalp voltage maps projected in 2D
    
3. Evaluate the plots to seek signs of non-brain activity 
    - This is the hardest part: experience and data quality are vital

4. Factor out (that is, exclude) non-brain components 

5. Reconstruct the data using only brain components 

Applications of ICA to neuroimaging data assume that brain signals are a linear combination of independent sources. In other words, they assume that the following idea holds true:

$$S^{(t)} = \alpha_1x^{(t)}_1 + \alpha_2x^{(t)}_2 + \alpha_3x^{(t)}_3 + ... + \alpha_nx^{(t)}_n$$

where:

- $S^{(t)}$ is the brain signal at time $t$
- $x^{(t)}_1, x^{(t)}_2, x^{(t)}_3, ..., x^{(t)}_n$ are $n$ independent factors, measured at time $t$. _Independent_ means that the value of any factor is the same regardless of the value of the others 
- $\alpha_1, \alpha_2, \alpha_3, ..., \alpha_n$ are the _weights_ of the other factors, which represent how much each factor contributes to the final signal at time $t$

In the case of EEG, the factors can only be brain areas, eyes, muscles, and other noise sources. Therefore, the equation above translates into something like:

$$ EEG^{(t)} = \alpha_1\text{Area}^{(t)}_1 + \alpha_2\text{Area}^{(t)}_2 + \alpha_3\text{Eyeblinks}^{(t)} + \alpha_4\text{Heartbeat}^{(t)} + ... + \alpha_n\text{Muscle Contraction}^{(t)}$$

If you think about it, the independence assumption is deeply flawed, as different brain and body parts cannot be thought of as separate entities. 

In particular, it is true that: 

- Brain areas interact with each other
- Heartbeats modulate brain activity, and vice versa
- Muscle contractions can originate from the brain

Therefore, ICA shall be used with great care: while it can factor brain signals into _statistically_ independent components, those components do not necessarily make neuroscientific sense.

Perhaps the only EEG component that can be considered approximately independent from the others is the eyeblink: while it does originate from the brain, it has three characteristics that make it reasonable to factor out. Those are:

1. **Lack of connection with cognitive dynamics:** eyeblinks are usually an act of no cognitive significance, which we perform merely to protect our eyes from drying out 
2. **Relative amplitude:** the amplitude of eyeblink artefacts is much larger than the amplitude of brain signals, so they must be removed because they alter the signal significantly
3. **Lack of better methods:** currenly, the main alternative to factoring out eyeblinks with ICA is to discard the signal segments that contain the eyeblinks. If the eyeblinks are close to interesting brain responses, this can cause an undesirable loss of data

## **2.2. Fitting**

This is the first step of the ICA lifecycle, and its very heart: here, the EEG signal is factored into statistically independent components that can be recombined back into the signal.

You have seen before that EEG data are represented as a matrix with dimension `channels x time`. The set of statistically independent components found by ICA will also be a matrix, with as many rows as there are components and as many columns as there are time points (`components x time`).

The transformation of the EEG data matrix into a components matrix is done by [multiplication](https://learninglab.rmit.edu.au/maths-statistics/linear-algebra/matrices-getting-started/m3-matrix-multiplication/) with an _unmixing matrix_. The unmixing matrix cannot be _any_ matrix: it must be such that the resulting components are statistically independent of one another. 

The reverse transformation from components to EEG data is also done by matrix multiplication &mdash; specifically, by multiplying the matrix of components with another one called _mixing matrix_. 

<p align="center">
<img src="./files/ica-nutshell.png" width="1000"/>
</p>

The process of fitting ICA consists mainly in finding the _unmixing_ matrix: once that is found, the _mixing_ matrix can be derived from it as the one that inverts the unmixing. \
Simplifying for educational purposes, we can think about this as finding some coefficient $\alpha$, whose inverse will necessarily be $\dfrac{1}{\alpha}$:

$$ y = \alpha \cdot x \implies x = \dfrac{1}{\alpha} \cdot y $$

### ⚠️ **Beware of randomness**

Fitting an ICA is an iterative process where candidate matrices are tried and rejected until the right one is found &mdash; where _"right"_ means _"such that the resulting components are statistically independent of one another"_. Conceptually, this process articulates over the following steps:

1. Perform unmixing 
2. Measure the statistical independence of the resulting components
3. Adjust the the unmixing matrix to increase the statistical independence of components
4. Repeat steps 1-3 until reaching one of the following conditions:
    - Statistical independence is maximised
    - A predefined number of iterations has been reached

Basically, fitting an ICA is a sophisticated form of trial-and-error, where the fitting algorithm performs an ideal walk across the space of all possible unmixing matrices until it finds the right one. \
Importantly, the starting point of this walk is not always the same, as the first trial (Step 1 in the list above) is performed with a matrix of [pseudorandom](https://en.wikipedia.org/wiki/Pseudorandomness) values. Therefore, applying ICA twice to the same dataset might yield two different sets of components due to different values of the initial matrix, which result in different walks accross the space of solutions. The likelihood of encountering this issue increases if the fitting process is set to end upon reaching a predefined number of iterations rather than a true solution, as different paths might end in significantly different places after the same number of steps.     

To increase the reproducibility of ICA &mdash; that is, to avoid obtaining two different solutions from two different ICA runs &mdash; one can set a _random seed_. A random seed (also called _random state_) is a number that forms the basis for the generation of a set of pseudorandom numbers. By setting a random seed, one ensures that the starting point of the walk across the space of all possible unmixing matrices is always the same, therefore ensuring that the fitting process is reproducible. 

--- 

❗We have seen in class that setting a random seed ensures reproducibility _within_ a given computer, but does not ensure reproducibility _between_ different computers. In other words, all of us obtained consistent results from multiple ICA runs, but some of us obtained different decompositions than the others. The most likely explanation for this phenomenon is that different computers might find different solutions due to one or more of the following causes:

- Different [pseudorandom number generators](https://en.wikipedia.org/wiki/Pseudorandom_number_generator): different computers might use the random seed in different ways, thereby setting different starting points for ICA's walk despite using the same seed
- Different decimal approximations: different computers might represent decimal numbers to different degrees of precision, resulting in small differences that propagate along the fitting process until yielding significantly different results 

Once again, these issues can be particularly severe if the fitting process is set to end upon reaching a predefined number of iterations rather than a true solution.

 👉 **Computational Reproducibility: Good Practices for ICA \& Feasibility Trade-Offs**

The problem of obtaining consistent ICA decompositions across computers is an example of a computational reproducibility problem.

Computational reproducibility is the ability to reproduce the exact results of a computational procedure (for example, a computerised data-analysis pipeline). While it might seem trivial, computational reproducibility is a known issue in science and technology: to give a rough sense of its relevance, searching it on Google Scholar returns [more than one million results](https://scholar.google.com/scholar?hl=it&as_sdt=0%2C5&q=computational+reproducibility&btnG=) as of late 2025.  

Computational reproducibility is influenced by a myriad factors, and determining the exact causes behind a reproducibility failure can be very difficult &mdash; especially in the absence of detailed information about all the software and hardware involved in and around a given process. However, close adherence to a few good practices can increase the reproducibility of any process.

In the case of ICA, the good practices for computational reproducibility are:

- Set a random seed
- Ensure that the fitting process stops upon reaching a true solution. To this end, it is useful to:
    - Increase the maximum number of iterations, to make sure that the fitting does not stop too early (and therefore, too distant from a true solution)
    - Decrease the tolerance parameter. The tolerance parameter is a threshold for change between unmixing matrices: when the difference between two unmixing matrices from two consecutive attempts is less than the tolerance parameter, the fitting can be considered concluded because the fact that changes are getting very small suggests that the true solution is very close

Clearly, ICA fitting cannot go on forever, so one might be forced to make less iterations and/or keep the tolerance parameter to a relatively high value. In these cases like in all others, it is very important to document which values were used for the parameters, so readers of your work can relate your output to the context that generated it. 

ICA-specific practices must always add to more general-purpose reproducibility practices, like the consistency of software versions: if two users use different versions of a toolbox (like MNE-Python), they might use slightly different implementations of the same algorithm and therefore, inject differences between their workflows. The importance of consistency applies to a given toolbox as well as its dependencies, and that is why we all use the same `conda` environment in this course. 

If you want to know more about computational reproducibility, you can read section 5 of [Good Programming Practices](./docs/good-programming-practices.md).

### 💻 **The actual code**

The code below initializes an `ICA` object, which is available from the `preprocessing` module of the `mne` package. 

The ICA object is initialised with a random state of 0, which means that the initial values of the unmixing matrix will be generated by some complex operation applied on number 0 (see [Wikipedia](https://en.wikipedia.org/wiki/Pseudorandom_number_generator) for details on pseudorandom number generators). \
Beside the random seed, it is possible to set important parameters like the number of desired components (`n_components`, which is equal to the number of channels by default) or the maximum number of iterations (`max_iter`, which is `1000` by default). The complete list of possible parameters (and their description) is available from the official MNE-Python documentation at [this](https://mne.tools/stable/generated/mne.preprocessing.ICA.html#mne.preprocessing.ICA) link.

After initialising the ICA object and assigning it to variable `ica`, the code calls method `fit()` and gives it the post-filtering epochs as input. This basically means that the post-filtering epochs are concatenated along the temporal axis to yield a single EEG data matrix of shape `channels x time`, which is used to find the _unmixing_ matrix and the corresponding _mixing_ matrix. \
Note that this will not return the decomposed data: it will merely find the ICA solution, without applying it to the data yet.   

In [ ]:
ica = mne.preprocessing.ICA(random_state=0)
ica.fit(post_filtering_epochs)

The process of fitting ICA involves large matrix multiplications, which must be repeated for a number of iterations. This can be very long and time-consuming, depending on the computational power at hand and the size of the data. 

Luckily, it is possible to save the result of the fitting process and write it to a file, so a given dataset can be decomposed once and the result can be used forever without repeating the fitting process. This also opens the door to sharing the result of your ICA decomposition with colleagues, students and readers of your work.   

The following code does just that: it saves the fitted ICA to a file named `fitted_ica.fif` within directory `data`. The argument `overwrite=True` allows you to rewrite the existing file should you need to re-run the procedure.

In [ ]:
ica.save(fname="data/fitted_ica.fif", overwrite=True)

Pre-computed ICA solutions can then be read from file and assigned to a Python variable, as follows:

In [ ]:
ica = mne.preprocessing.read_ica(fname="data/fitted_ica.fif")

## **2.3. Components Selection**

Once the data are factored into a set of components, we can plot those components in a variety of formats. The aim is to evaluate their appearance and classify them as cerebral activity versus muscular, ocular, cardiac, or environmental noise.

One way to visualize components is to plot their scalp topography &mdash; that is, a colour-coded map of how component intensity distributes over the scalp. Usually, strongly positive activity values are red in a scalp topography, while strongly positive values are blue and intermediate values have intermediate colours like shades of orange, yellow and green.

Scalp topographies are usually accompanied by a series of supporting plots, including:

- The component's power spectrum
- The component's event-related potential (ERP) &mdash; that is, its average across epochs 
- The component's ERP image &mdash; that is, its ERP accompanied by a colour-coded representation of all single trials

All these plots are relevant to evaluate a component, as they all provide complementary information that, taken together, can build a complete picture.

We are especially interested in finding and rejecting ocular components, which are usually revealed by the following set of information:

- Intense activity near the eyes. In particular, it is usually true that:

    - A unipolar concentration of activity around the sagittal midline suggests _vertical_ eye movements
    - A left-right bipolar concentration of activity suggests _horizontal_ eye movements
    
- Highest power at lower frequencies (that is, 5 Hz and below)

- Distribution that is random over trials, or stimulus-locked in case of stimuli that can elicit eye movements (e.g., TMS pulses can elicit blinks)

The code below calls method `plot_components()` on the `ica` object, using the post-filtering epochs as data of interest and picking only the first 30 components (in order of variance explained) for brevity.

This will generate one scalp topography for each component. Clicking once on the scalp topography will open a new panel that displays the topography alongside all other plots &mdash; including the power spectrum, the ERP and the ERP image.

Note that calling `plot_components()` will apply the ICA solution that was previously found by `fit()`, but only for visualization purposes: the aim is only for users to _inspect_ components and decide which ones are worth rejecting.  

In [ ]:
ica.plot_components(inst=post_filtering_epochs, picks=range(30))

A second (and complementary) method to evaluate independent components is `plot_sources()`, which plots the time series of components. 

Visualizing time series is useful because it gives a sense of how components behave in time. For example, eye components are mostly stationary except for brief, large-amplitude spikes (for blinks) or step curves of varying length (for horizontal movements). 

In [ ]:
ica.plot_sources(inst=post_filtering_epochs)

After evaluating the components, it is possible to reject them by assigning a list of their numbers to the `exclude` attribute of the `ica` object. For example, running `ica.exclude = [0,1,2]` will reject components number 0, 1, and 2 (that is, those who explain the most, second-most, and third-most variance).

The code in the following cell opens a pop-up window where you can insert the numbers of interest, then assigns the resulting list to variable `components_to_reject`. In the next cell, `components_to_reject` is assigned to `ica.exclude` and the components are rejected using method `apply()`. Finally, epochs are averaged and plotted as usual. 

⚠️ Note that `components_to_reject = utils.select_items(item_type="ica")` might not work on Mac computers, probably due to compatibility issues between the Python library used to build the pop-up window and Mac graphics. If you encounter this issue, do something like `ica.exclude = [0,1,2]` instead (numbers are just an example).

In [ ]:
components_to_reject = utils.select_items(item_type="ica")

In [ ]:
ica.exclude = components_to_reject
ica.apply(post_filtering_epochs.load_data())
post_ica_epochs = post_filtering_epochs
post_ica_tep = post_ica_epochs.average()
post_ica_tep.plot();

# **3. Manual Artifact Rejection**

## **3.1. Rationale**

Manual artifact rejection (MAR) is the act of visually inspecting EEG data and flagging epochs for rejection with a mouse click. Afer researchers have inspected the whole dataset, flagged epochs are discarded. 

MAR is the simplest form of data cleaning, as it merely thrashes unusable data. It has mainly three use cases:

- Remove the largest artifacts before feeding the data to automatic procedures like ICA
    - This can increase the effectiveness of automatic procedures, for example yielding clearer ICA decompositions

- Remove leftover artifacts after automatic procedures
    - For example, one may remove ocular artifacts by rejecting ICA's ocular components, but that will not remove the artifacts due to occasional muscle contractions

- Remove the most obvious artifacts, and do no more
    - This is what purists do: no automatic procedures, only MAR (plus some filtering and bad channels rejection at most)

In all three cases, the criteria to follow for MAR are usually the following:

- The data are characterised by artifacts that present as high-frequency, high-amplitude fluctuations that cannot plausibly be cerebral
- The artifacts in question are circumscribed in time
- The artifacts in question affect multiple channels

The amount of epochs that can be rejected depends on the amount of epochs at hand: MAR must hit a trade-off between cleaning the data as much as possible and preserving as many epochs as possible to avoid losing statistical power. There is no universally valid number, but as a rule of thumb it would be better to keep at least 100 epochs per condition. This goal is usually easy to reach, as the number of epochs that are worth rejecting manually are usually around ten (although that depends a lot on the quality of the data).

## **3.2. Execution**

To do MAR in MNE-Python, it is sufficient to plot the epoched data and click on the artifactual epochs once. Clicked epochs will turn red and that will mean that they were added to a "bads" list*. \
Epochs added to a "bads" list do not count for subsequent operations, meaning that they will not be fed to any downstream preprocessing step nor taken into account in the computation of evoked potentials.

Besides plotting and clicking, MNE-Python provides at least one other way to perform MAR: the `drop_bad()` function. Instead of adding them to a "bads" list, this will actually delete the epochs from Python's memory (but not from your _computer's_ memory). This can be inconvenient because if you change your mind about one epoch, you cannot go back with ease: you need to recreate the variable at the state where it last had all the epochs, which might entail re-running one or more previous steps.**

---
*_Similarly, clickling on the label of a channel adds the channel to the "bads" list_

**_In this respect, it helps a lot to save intermediate preprocessing outputs in dedicated files_

The code below plots `post_ica_epochs`, allowing users to scroll through them and flag them for rejection if necessary:

In [ ]:
post_ica_epochs.plot()

After rejecting one or more epochs, you can verify that they have excluded from the data of interest by printing the dataset's info: the `Events count` field will have decreased from the initial value (in our case, 200) to any other smaller value.

In [ ]:
post_ica_epochs

# **4. Assessing & Computing a Transcranial Evoked Potential (TEP)**

At the end of a preprocessing pipeline, you look at the result and assess if it contains signs of the brain activity of interest. 

In the case of an event-related design, the brain activity of interest is an evoked potential &mdash; in our case, a _transcranial_ evoked potential (TEP), which is a fluctuation of the EEG signal evoked by a TMS pulse.

The computation of the TEP and its assessment can be preceded by a short sequence of preparatory steps. These last steps aim at removing some more noise and converting the data to a format where the evoked activity can stand out from the background. 

In the publication that generated the data that we used, the last steps are:

- Low-pass filtering with a cut frequency of 70 Hz
- Rereferencing to the average reference
- Baseline correction

## **4.1. Preparatory Steps**

#### **Low-pass filtering** 

The code below applies a low-pass filter to the epoched data, with a cut frequency of 70 Hz. 

This means that all frequencies above 70 Hz will be attenuated, and all frequencies below 70 will remain as they are. This is useful because frequencies below 70 Hz are usually not of physiological interest, as they mostly generated by muscle contractions and environmental noise. 

In [ ]:
low_pass_filtered_data = mne.filter.filter_data(data=post_ica_epochs.get_data(),
                                                sfreq=post_ica_epochs.info["sfreq"],
                                                l_freq=None,
                                                h_freq=70.0,
                                                method="iir",
                                                iir_params=None,
                                                copy=True,
                                                phase="zero")
low_pass_filtered_epochs = mne.EpochsArray(data=low_pass_filtered_data,
                                           info=post_ica_epochs.info,
                                           baseline=None)
low_pass_filtered_tep = low_pass_filtered_epochs.average()
low_pass_filtered_tep.plot();

#### **Rereferencing**

Rereferencing is a common step in EEG preprocessing, and it is a very simple one: you just take one channel (the new reference) and subtract it to all others. 

Data enter any preprocessing pipeline with a built-in reference, which is the one applied online. If the reference electrode is strongly localised, signals from distant electrodes will be over-estimated with respect to neighbouring ones, creating a false impression of activity differences between the underlying generators. For example:

- Choosing an occipital reference like `Oz` will emphasize frontal signals, creating the false impression that frontal generators are more active than occipital ones
- Choosing a frontal reference like `AFz` will emphasize occipital signals, creating the false impression that occipital generators are more active than frontal ones
- Choosing a left-lateralised reference like `TP9` will emphasize right-hemisphere signals, creating the false impression that right generators are more active than left ones
- Choosing a right-lateralised reference like `TP10` will emphasize left-hemisphere signals, creating the false impression that left generators are more active than right ones

For this reason it is very important to choose a neutral reference &mdash; that is, one that does not over- or underestimate any signals. 

The average between the TP9 and TP10 electrodes (also called _digitally linked mastoids &mdash; DLM_) is widely considered to be a neutral reference, as it meets three important criteria:

- TP9 and TP10 are approximately halfway between the front and back of the head, so there is no risk of over- or underestimating frontal or occipital signals
- The reference is neither TP9 (which is left-lateralised) nor TP10 (which is right-lateralised), but their average. Therefore, it can be thought of as the midpoint between left and right
- TP9 and TP10 are located on the left and right mastoids, respectively. These are two bones, so they are approximately neutral from an electrophysiological point of view. Therefore, using them as reference (which implies losing their data) entails no loss of interesting information 

The online reference for our data was indeed the average of TP9 and TP10, so it was a neutral one. Nonetheless, the authors of the study chose to rereference to the average of all channels. 

The code below sets an average reference to the epoched data, using the `set_eeg_reference()` function provided by MNE. 

As can be seen, `set_eeg_reference()` returns two objects: the actual rereferenced data and the reference itself &mdash; which, in this case, is the reference of all channels.  

In [ ]:
rereferenced_epochs, reference = mne.set_eeg_reference(inst=low_pass_filtered_epochs,
                                                       ref_channels="average") 

#### **Baseline correction** 

Baseline correction is the act of substracting a pre-stimulus mean to the post-stimulus signal, in order to represent the post-stimulus signal as a deviation from a baseline. 

The authors of the data-generating study computed the pre-stimulus mean between 100 ms pre-stimulus and 2 ms pre-stimulus. Therefore, the code below localizes those points on the temporal axis of the data.

In [ ]:
TIME_OF_EVENT = 1.1                             # seconds
baseline_window_start = TIME_OF_EVENT - 0.100   # seconds
baseline_window_end = TIME_OF_EVENT - 0.002     # seconds

Having defined the start and end of the baseline window, we can use it for baseline correction and, subsequenly, compute a TEP for visualization as usual. The code below does just that:

In [ ]:
baseline_corrected_epochs = rereferenced_epochs.apply_baseline(baseline=(baseline_window_start, baseline_window_end))
baseline_corrected_tep = baseline_corrected_epochs.average()
baseline_corrected_tep.plot();

## **4.2. Computation of the TEP** 

This is the simplest part, as an evoked potential is merely an average over epochs. The code below computes such average after cropping the pre-stimulus window to 100 ms for more visual clarity. 

In [ ]:
final_epochs = baseline_corrected_epochs.crop(tmin=1.0,
                                              tmax=1.6,
                                              include_tmax=True)
final_tep = final_epochs.average()
final_tep.plot();